# 情緒分析 - 使用OpenAI API
使用 OpenAI API 根據資料框 '內容' 欄位的文字內容預測情緒。

In [13]:
import pandas
# 載入資料集
df = pandas.read_csv('https://raw.githubusercontent.com/ywchiu/hp_ai_advanced_training/refs/heads/main/data/hp_reviews.csv')
# 顯示資料集前幾列
df.head()

,內容,情緒
0,HP就是穩！大廠牌品管有保證啦～不像某些雜牌三不五時就出包，我公司採購都指定HP或Dell，...,正面
1,原廠服務真的讚，上次要加記憶體，送去服務中心免費裝，反觀國產牌還要收費= = 而且HP的工程...,正面
2,買了Victus 15用來打game，外型低調不會被看出是電競筆電，爽！在咖啡廳用也不會太招...,正面
3,OmniBook超薄超輕，帶出門完全沒負擔，電池還能撐一整天👍 我每天通勤背著也不會肩膀痠，...,正面
4,黑五搶到HP 14吋只要236美金！！學生黨表示太香了～～ 規格雖然不是頂級但日常使用綽綽有...,正面


## 設定 openai api

安裝必要的函式庫並設定您的 OpenAI API 金鑰。


In [14]:
%pip install openai

import os
from google.colab import userdata

# 將 "YOUR_OPENAI_API_KEY" 替換成您的實際 API 金鑰
# 建議將您的 API 金鑰儲存在環境變數中
# 並使用 os.getenv('OPENAI_API_KEY') 來存取
# 在此僅為示範目的直接指定
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## 定義情緒預測函式

### Subtask:
建立一個函式，該函式接收文字輸入並使用 OpenAI API 預測情緒。


In [22]:
SYSTEM_PROMPT = '''
你是一個情感分析模型。請判斷以下文字的情緒是 '正面'、'中立'或 '負面'。用json回覆 {"emotion": "正面"} 、 {"emotion": "中立"} 或 {"emotion": "負面"}。

根據提供的文字，分析情緒並得出結論。

# Output Format

請確保回答格式為 JSON，僅包含 'emotion' 作為鍵名稱，例如：{"emotion": "正面"} 或 {"emotion": "負面"}。

# Examples

**Example 1:**
- Input: "天气真好，我感到非常愉快。"
- Output: {"emotion": "正面"}

**Example 2:**
- Input: "今天的服务太糟糕了，我非常失望。"
- Output: {"emotion": "負面"}

# Notes

在判斷時請考慮語境和詞彙的積極或消極性。

'''

In [25]:
from openai import OpenAI
import json

client = OpenAI()

def predict_sentiment(text):
  """使用 OpenAI API 預測情緒。"""
  try:
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": SYSTEM_PROMPT
                    }
                ]
            },
            {"role": "user", "content": text}
        ],
        response_format={
            "type": "json_object"
        },
        temperature=1,
        max_tokens=10, # 由於預期輸出為簡短的 JSON，因此將 max_tokens 設定得較小
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    sentiment_json = json.loads(response.choices[0].message.content.strip())
    sentiment = sentiment_json.get("emotion")
    # 確保回覆是預期的情緒之一
    if sentiment in ["正面","中立", "負面"]:
        return sentiment
    else:
        # 如果模型給出了意外的回覆，回傳一個佔位符或作為錯誤處理
        print(f"API 回覆意外：{sentiment}")
        return "無法預測" # 或拋出錯誤
  except Exception as e:
    print(f"發生錯誤：{e}")
    return "無法預測" # 處理 API 錯誤

## 應用函式至DataFrame

將定義的函式應用到DataFrame的 '內容' 欄位，為每條評論預測情緒。


In [26]:
from tqdm.auto import tqdm

# 為 apply 函式加入進度條
tqdm.pandas()

# 使用進度條將 predict_sentiment 函式應用到 '內容' 欄位
df['預測情緒'] = df['內容'].progress_apply(predict_sentiment)

# 顯示帶有新的 '預測情緒' 欄位的資料框
display(df.head())

  0%|          | 0/150 [00:00<?, ?it/s]

,內容,情緒,預測情緒
0,HP就是穩！大廠牌品管有保證啦～不像某些雜牌三不五時就出包，我公司採購都指定HP或Dell，...,正面,正面
1,原廠服務真的讚，上次要加記憶體，送去服務中心免費裝，反觀國產牌還要收費= = 而且HP的工程...,正面,正面
2,買了Victus 15用來打game，外型低調不會被看出是電競筆電，爽！在咖啡廳用也不會太招...,正面,正面
3,OmniBook超薄超輕，帶出門完全沒負擔，電池還能撐一整天👍 我每天通勤背著也不會肩膀痠，...,正面,正面
4,黑五搶到HP 14吋只要236美金！！學生黨表示太香了～～ 規格雖然不是頂級但日常使用綽綽有...,正面,正面


## 詳細評估預測結果

計算並顯示每個情緒類別 (正面、中立、負面) 的 Precision, Recall, 和 F1 Score。

In [28]:
from sklearn.metrics import classification_report

# 生成分類報告
report = classification_report(df['情緒'], df['預測情緒'])

print(report)

              precision    recall  f1-score   support

          中立       1.00      0.84      0.91        50
          正面       0.84      0.98      0.91        50
          負面       0.98      0.98      0.98        50

    accuracy                           0.93       150
   macro avg       0.94      0.93      0.93       150
weighted avg       0.94      0.93      0.93       150



# 情緒分析 - -Embedding
利用 OpenAI 的 `text-embedding-3-large` 模型產生文字嵌入，並使用 SVM 分類器對情緒進行分類，最後評估分類器的效能，考量正面、中立、負向的 precision, recall, 和 f1 score。

## 產生文字嵌入


使用 OpenAI 的 `text-embedding-3-large` 模型為DataFrame '內容' 欄位的文字產生嵌入向量。


In [34]:
from tqdm.auto import tqdm

# 為 apply 函式加入進度條
tqdm.pandas()

def get_embedding(text):
  """使用 OpenAI 的 text-embedding-3-large 模型產生文字嵌入。"""
  try:
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=text,
    )
    return response.data[0].embedding
  except Exception as e:
    print(f"生成嵌入向量時發生錯誤：{e}")
    return None # 發生錯誤時回傳 None

# 使用進度條將 get_embedding 函式應用到 '內容' 欄位
df['嵌入向量'] = df['內容'].progress_apply(get_embedding)

# 顯示帶有新的 '嵌入向量' 欄位的資料框前幾列
display(df.head())

  0%|          | 0/150 [00:00<?, ?it/s]

,內容,情緒,預測情緒,嵌入向量
0,HP就是穩！大廠牌品管有保證啦～不像某些雜牌三不五時就出包，我公司採購都指定HP或Dell，...,正面,正面,"[0.0216562207788229, 0.006734313443303108, -0...."
1,原廠服務真的讚，上次要加記憶體，送去服務中心免費裝，反觀國產牌還要收費= = 而且HP的工程...,正面,正面,"[0.035742394626140594, -0.007179505191743374, ..."
2,買了Victus 15用來打game，外型低調不會被看出是電競筆電，爽！在咖啡廳用也不會太招...,正面,正面,"[0.02243438921868801, 0.004921004641801119, -0..."
3,OmniBook超薄超輕，帶出門完全沒負擔，電池還能撐一整天👍 我每天通勤背著也不會肩膀痠，...,正面,正面,"[0.0221847016364336, 0.00034546686219982803, -..."
4,黑五搶到HP 14吋只要236美金！！學生黨表示太香了～～ 規格雖然不是頂級但日常使用綽綽有...,正面,正面,"[-0.00874573364853859, 0.009696762077510357, -..."


## 準備 svm 分類器的資料

將產生的嵌入向量和對應的情緒標籤準備成 SVM 分類器所需的輸入格式。


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 提取特徵資料 (X) 和目標標籤 (y)
X = np.array(df['嵌入向量'].tolist())
y = df['情緒']

# 將文字標籤轉換為數值
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded)

print("特徵資料 (X) 形狀:", X.shape)
print("目標標籤 (y) 形狀:", y_encoded.shape)
print("訓練集特徵資料 (X_train) 形狀:", X_train.shape)
print("測試集特徵資料 (X_test) 形狀:", X_test.shape)
print("訓練集目標標籤 (y_train) 形狀:", y_train.shape)
print("測試集目標標籤 (y_test) 形狀:", y_test.shape)

特徵資料 (X) 形狀: (150, 3072)
目標標籤 (y) 形狀: (150,)
訓練集特徵資料 (X_train) 形狀: (112, 3072)
測試集特徵資料 (X_test) 形狀: (38, 3072)
訓練集目標標籤 (y_train) 形狀: (112,)
測試集目標標籤 (y_test) 形狀: (38,)


## 訓練 svm 分類器


使用準備好的資料訓練一個 SVM 分類器。


In [40]:
from sklearn.svm import SVC

# Create an SVC classifier instance
svm_classifier = SVC(kernel = 'rbf')

# Train the model using the training data
svm_classifier.fit(X_train, y_train)

print("SVM classifier trained successfully.")

SVM classifier trained successfully.


## 評估 svm 分類器

評估訓練好的 SVM 分類器在情緒分類上的效能。


In [41]:
from sklearn.metrics import classification_report

# 使用訓練好的 SVM 分類器在測試集上進行預測
y_pred = svm_classifier.predict(X_test)

# 使用 classification_report 函式評估模型效能
# 使用 label_encoder.classes_ 來顯示正確的類別名稱
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

# 印出分類報告
print(report)

              precision    recall  f1-score   support

          中立       0.85      0.92      0.88        12
          正面       0.92      0.85      0.88        13
          負面       1.00      1.00      1.00        13

    accuracy                           0.92        38
   macro avg       0.92      0.92      0.92        38
weighted avg       0.92      0.92      0.92        38



## 總結分析

本次情緒分析實驗使用了兩種方法來預測評論的情緒：

1.  **使用 OpenAI Chat Completion API (gpt-4.1-mini)**：直接請模型判斷評論是「正面」、「中立」或「負面」。
2.  **使用 OpenAI Embedding API (text-embedding-3-large) 結合 SVM 分類器**：先將評論轉換為向量表示，再使用 SVM 模型進行分類。

以下是兩種方法的結果比較：

### OpenAI Chat Completion API (gpt-4.1-mini) 結果

| 情緒   | Precision | Recall | F1-Score | Support |
| :----- | :-------- | :----- | :------- | :------ |
| 中立   | 1.00      | 0.84   | 0.91     | 50      |
| 正面   | 0.84      | 0.98   | 0.91     | 50      |
| 負面   | 0.98      | 0.98   | 0.98     | 50      |
| **總體** | **0.94**  | **0.93** | **0.93** | **150** |

**分析**：

*   Chat Completion API 在負面情緒的預測上表現出色，Recall 和 F1-Score 都達到了 0.98。
*   正面情緒的 Recall 很高 (0.98)，但 Precision 稍低 (0.84)，這表示模型傾向於將一些非正面評論誤判為正面。
*   中立情緒的 Precision 為 1.00，但 Recall 較低 (0.84)，表示模型在辨識中立評論時很精確，但可能會遺漏一些中立評論。

### OpenAI Embedding API (text-embedding-3-large) + SVM 結果

| 情緒   | Precision | Recall | F1-Score | Support |
| :----- | :-------- | :----- | :------- | :------- |
| 中立   | 0.85      | 0.92   | 0.88     | 12       |
| 正面   | 0.92      | 0.85   | 0.88     | 13       |
| 負面   | 1.00      | 1.00   | 1.00     | 13       |
| **總體** | **0.92**  | **0.92** | **0.92** | **38**   |

**分析**：

*   Embedding + SVM 方法在負面情緒的預測上同樣表現優異，Precision、Recall 和 F1-Score 均為 1.00。
*   正面和中立情緒的 Precision 和 Recall 相對平衡，F1-Score 均為 0.88。
*   需要注意的是，Embedding + SVM 的評估是基於測試集 (38 筆資料)，而 Chat Completion API 是基於整個資料集 (150 筆資料)。雖然兩者在整體 accuracy 和 macro/weighted avg 的 F1-Score 相近 (0.93 vs 0.92)，但基於不同資料集大小的直接比較需要謹慎。

### 結論

兩種方法在情緒分析任務上都取得了不錯的成效，特別是在負面情緒的辨識上表現良好。

*   **Chat Completion API** 的優點在於直觀且易於實施，特別適合需要快速原型驗證或處理少量資料的場景。它在正面情緒的 Recall 上表現突出。
*   **Embedding + SVM** 的優點在於利用了文字的向量表示，可以捕捉更豐富的語義信息，並透過訓練分類器來適應特定任務。它在負面情緒的 Precision 和 Recall 上都達到了完美。

選擇哪種方法取決於具體的應用需求。如果追求最高的負面情緒辨識準確率，Embedding + SVM 可能更適合；如果需要快速且全面的情緒分類，Chat Completion API 是一個不錯的選擇。

總體而言，本次實驗顯示 OpenAI 的模型在中文情緒分析任務上具有很高的潛力。